In [68]:
import sys
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.models import *
from keras.layers import *

%matplotlib inline

coffee = pd.read_csv('./updated_coffee_df.csv')
coffee.head()



coffee['Date'] = pd.to_datetime(coffee['Date'])
coffee = coffee.set_index("Date")

coffee_shift_df = pd.concat([coffee[['Title']].shift(1),coffee[['Rate_of_Change']].shift(1), coffee[['Price']]],axis=1)
coffee_shift_df.dropna(inplace=True)
coffee_shift_df.head()

,Title,Rate_of_Change,Price
Date,,,
2007-01-03,India earns more from higher coffee exports in...,0.000000,1.1760
2007-01-04,Friesland raises stake in Indonesian subsidiar...,0.022075,1.1451
2007-01-05,Nymex announces start date for soft commodity ...,-0.026276,1.1506
2007-01-07,India's largest coffee chain extends to Pakistan,0.004803,1.1506
2007-01-09,Honduran coffee sales Ugandan coffee funds Soy...,0.000000,1.1290


In [69]:
tokenizer = Tokenizer()

X = coffee_shift_df['Title']
y = coffee_shift_df['Price']

X_token = tokenizer.fit_on_texts(X)
X_matrix = tokenizer.texts_to_matrix(X)
vocab_size = len(tokenizer.word_index) + 1

In [77]:
input_tensor = Input((3,1))
print(input_tensor)
X = input_tensor



Tensor("input_20:0", shape=(?, 3, 1), dtype=float32)


In [78]:
for i in range(2):
    X = Bidirectional(SimpleRNN(32 * 2** i, return_sequences=True))(X)
    X = Activation('relu')(X)
    
X = Bidirectional(SimpleRNN(512, return_sequences=False))(X)
X = Activation('relu')(X)
X = Dense(1, activation = 'linear')(X)
X = PReLU(alpha_initializer ='zeros',
         alpha_regularizer = None,
         alpha_constraint = None,
         shared_axes = None)(X)
model = Model(inputs=input_tensor, output=X)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("p_...)`
  if sys.path[0] == '':


In [79]:
model.compile(loss = 'mse', optimizer='adam')
model.fit(X_matrix, y, epochs=20, validation_split=.20, batch_size=100)

ValueError: Error when checking input: expected input_20 to have 3 dimensions, but got array with shape (1464, 3822)